# Top Chef Contentestants

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

In [2]:
#this is a list of all of the Top Chef contestants
url = 'https://en.wikipedia.org/wiki/List_of_Top_Chef_contestants'

In [3]:
#use pandas to read the table
tables = pd.read_html(url)
tables

[                     Name   Age         Place of Residence             Season  \
 0                 Ken Lee  31.0    Los Angeles, California  1 (San Francisco)   
 1         Cynthia Sestito  52.0     East Hampton, New York  1 (San Francisco)   
 2              Brian Hill  37.0    Los Angeles, California  1 (San Francisco)   
 3           Candice Kumai  23.0       Pasadena, California  1 (San Francisco)   
 4              Lisa Parks  45.0    Los Angeles, California  1 (San Francisco)   
 ..                    ...   ...                        ...                ...   
 280       Adrienne Wright   NaN      Boston, Massachusetts      16 (Kentucky)   
 281       Michelle Minori   NaN  San Francisco, California      16 (Kentucky)   
 282         Eric Adjepong   NaN             Washington, DC      16 (Kentucky)   
 283  Kelsey Barnard Clark   NaN            Dothan, Alabama      16 (Kentucky)   
 284          Sara Bradley   NaN          Paducah, Kentucky      16 (Kentucky)   
 
      No. of Q

In [4]:
#read the table into a data frame
df = tables[0]
df.head()

,Name,Age,Place of Residence,Season,No. of QFW1,No. of EW2,Ranked
0,Ken Lee,31.0,"Los Angeles, California",1 (San Francisco),0,0,12th
1,Cynthia Sestito,52.0,"East Hampton, New York",1 (San Francisco),0,0,11th3
2,Brian Hill,37.0,"Los Angeles, California",1 (San Francisco),0,0,10th
3,Candice Kumai,23.0,"Pasadena, California",1 (San Francisco),0,1,9th
4,Lisa Parks,45.0,"Los Angeles, California",1 (San Francisco),0,1,8th


In [5]:
#keep just the variables we want
cols=['Name','Season','No. of QFW1','No. of EW2','Ranked','Place of Residence']
contestants=df[cols]

#rename columns
contestants=contestants.rename(columns={'Name':'chef_name',
                                        'No. of QFW1':'qf_wins',
                                       'No. of EW2':'ew_wins',
                                       'Place of Residence':'home',
                                       'Ranked':'ranked'})

#split the season into a season number and a location
contestants['season_num']=contestants['Season'].str.split(expand=True)[0].str.strip()

#split the contestants home location in to city and state
contestants['home_city']=contestants['home'].str.split(',',expand=True)[0].str.strip()
contestants['home_state']=contestants['home'].str.split(',',expand=True)[1].str.strip()

#strip blank characters from contestants name and ranked
contestants['chef_name']=contestants['chef_name'].str.strip()
contestants['ranked']=contestants['ranked'].str.strip()

#drop columns we no longer need  
contestants_df=contestants.drop(columns=['Season','home'])

#clean up some 'dirty' data entries
contestants_df['ranked']=contestants_df['ranked'].str.split('30',expand=True)[0].str.strip()
contestants_df['chef_name']=contestants_df['chef_name'].str.split('Ranked',expand=True)[0].str.strip()

#create a chef_id that strips spaces for better merge results
contestants_df['chef_id']=contestants_df['chef_name'].str.replace(' ','')

#take a look at the winners
contestants_df.loc[contestants_df['ranked']=="Winner"]


,chef_name,qf_wins,ew_wins,ranked,season_num,home_city,home_state,chef_id
11,Harold Dieterle,2,1,Winner,1,New York City,New York,HaroldDieterle
26,Ilan Hall,1,2,Winner,2,New York City,New York,IlanHall
41,Hung Huynh,4,1,Winner,3,Las Vegas,Nevada,HungHuynh
57,Stephanie Izard,2,4,Winner,4,Chicago,Illinois,StephanieIzard
74,Hosea Rosenberg,1,2,Winner,5,Boulder,Colorado,HoseaRosenberg
91,Michael Voltaggio,2,3,Winner,6,Pasadena,California,MichaelVoltaggio
108,Kevin Sbraga,1,1,Winner,7,Willingboro,New Jersey,KevinSbraga
126,Richard Blais,4,4,Winner,8,Atlanta,Georgia,RichardBlais
155,Paul Qui,2,8,Winner,9,Austin,Texas,PaulQui
179,Kristen Kish,1,4,Winner,10,Boston,Massachusetts,KristenKish


In [27]:
#create a season table that lists the season number and season location
cols=['Season']
season=df[cols].copy()

#split the season variable into a number and location
season['season_num']=season['Season'].str.split(expand=True)[0].str.strip()
season['season_city']=season['Season'].str.split('(',expand=True)[1].str.strip()
season['season_city']=season['season_city'].str[0:-1].str.strip()

#keep one entry per season
group_df=season.groupby(by="season_num")
season_df=group_df.first()

#drop the original season column that we no longer need
season_df=season_df.drop(columns=['Season'])
season_df


,season_city
season_num,
1,San Francisco
10,Seattle
11,New Orleans
12,Boston
13,California
14,Charleston
15,Colorado
16,Kentucky
2,Los Angeles


# Top Chef Restaurants
Data for this section came from a reddit post. Reddit posts cannot be scraped, so I copied and pasted it to Excel. I did some reformatting there, including using VBA to pull the URLs from the hyperlinks into their own columns. More than one Top Chef can own a restaurant (there are several partnerships) and there are a few chains where restaurants of the same name exist in different cities with different URLs

In [7]:
#read in CSV
rest=pd.read_csv('TopChef_restaurants.csv')
rest.head()

,Location,Restaurant,URL,Chef,Season
0,"Chicago, IL",The Dearborn,http://www.thedearborntavern.com/,Aaron Cuschieri,S11
1,"New York, Ny",A&E Supply Co.,http://www.aesupplyco.com/,Adam Harvey,S12
2,"Los Angeles, Ca",Cascabel,http://cascabelrestaurant.com/cms/,Alex Eusebio,S5
3,"Los Angeles, Ca",Sweetsalt,http://sweetsaltfood.com/,Alex Eusebio,S5
4,"Los Angeles, Ca",Ditmas,http://www.ditmasla.com/,Alex Reznik,S7


In [8]:
#split the restaurant location into city and state
rest['rest_city']=rest['Location'].str.split(',',expand=True)[0].str.strip()
rest['rest_state']=rest['Location'].str.split(',',expand=True)[1].str.strip()
rest['rest_state']=rest['rest_state'].str.upper()

#rename the columsn
rest=rest.rename(columns={'URL':"rest_url",
                          'Chef':"chef_name",
                          'Restaurant':'rest_name'})

#drop the columsn we no longer need--Season variable here is dirty with multiple seasons in one entry
restaurant=rest.drop(columns=['Location','Season'])

#create the chef_id to merge to the contestants table
restaurant['chef_name']=restaurant['chef_name'].str.strip()
restaurant['chef_id']=restaurant['chef_name'].str.replace(' ','')
restaurant.head()

,rest_name,rest_url,chef_name,rest_city,rest_state,chef_id
0,The Dearborn,http://www.thedearborntavern.com/,Aaron Cuschieri,Chicago,IL,AaronCuschieri
1,A&E Supply Co.,http://www.aesupplyco.com/,Adam Harvey,New York,NY,AdamHarvey
2,Cascabel,http://cascabelrestaurant.com/cms/,Alex Eusebio,Los Angeles,CA,AlexEusebio
3,Sweetsalt,http://sweetsaltfood.com/,Alex Eusebio,Los Angeles,CA,AlexEusebio
4,Ditmas,http://www.ditmasla.com/,Alex Reznik,Los Angeles,CA,AlexReznik


In [9]:
#more than one chef can be associated with more than one restaurant. We need a crosswalk table.
#keep rows the uniquely identifty chef and restaurant combinations, drop any duplicates

cr1_df=restaurant[['chef_id','rest_name','rest_city']]
cr1_df.head()
cr1_df=cr1_df.drop_duplicates(ignore_index=True)


In [10]:
#Create a restaurant ID field for each restaurant, location
restaurant_df=restaurant.drop(columns=['chef_id','chef_name'])
restaurant_df=restaurant_df.drop_duplicates(ignore_index=True)
restaurant_df.reset_index(inplace=True)
restaurant_df=restaurant_df.rename(columns={'index':'rest_id'})
restaurant_df.head()

,rest_id,rest_name,rest_url,rest_city,rest_state
0,0,The Dearborn,http://www.thedearborntavern.com/,Chicago,IL
1,1,A&E Supply Co.,http://www.aesupplyco.com/,New York,NY
2,2,Cascabel,http://cascabelrestaurant.com/cms/,Los Angeles,CA
3,3,Sweetsalt,http://sweetsaltfood.com/,Los Angeles,CA
4,4,Ditmas,http://www.ditmasla.com/,Los Angeles,CA


In [11]:
#merge restaurant ID into the chef-restaurant crosswalk file
cols=['rest_name','rest_city']
chef_rest_df=pd.merge(cr1_df,restaurant_df,how='outer',on=cols)
chef_rest_df=chef_rest_df[['chef_id','rest_id']]
chef_rest_df.head()

,chef_id,rest_id
0,AaronCuschieri,0
1,AdamHarvey,1
2,AlexEusebio,2
3,AlexEusebio,3
4,AlexReznik,4


In [12]:
#connect to postgres database chefs_db.  See TopChef.sql for db specifications
connection_string = "postgres:postgres@localhost:5432/chefs_db"
engine = create_engine(f'postgresql://{connection_string}')

In [13]:
#check to see if we have all tables
engine.table_names()

['restaurants', 'chef_rest', 'season', 'contestants']

In [17]:
#export dataframes to postgres
season_df.to_sql(name='season', con=engine, if_exists='append', index=True)
contestants_df.to_sql(name='contestants', con=engine, if_exists='append', index=False)
restaurant_df.to_sql(name='restaurants', con=engine, if_exists='append', index=False)
chef_rest_df.to_sql(name='chef_rest', con=engine, if_exists='append', index=False)

In [18]:
#check files in postgres
pd.read_sql_query('select * from contestants', con=engine).head()

,chef_name,chef_id,qf_wins,ew_wins,ranked,season_num,home_city,home_state
0,Ken Lee,KenLee,0,0,12th,1,Los Angeles,California
1,Cynthia Sestito,CynthiaSestito,0,0,11th3,1,East Hampton,New York
2,Brian Hill,BrianHill,0,0,10th,1,Los Angeles,California
3,Candice Kumai,CandiceKumai,0,1,9th,1,Pasadena,California
4,Lisa Parks,LisaParks,0,1,8th,1,Los Angeles,California


In [19]:
pd.read_sql_query('select * from restaurants',con=engine).head()

,rest_id,rest_name,rest_url,rest_city,rest_state
0,0,The Dearborn,http://www.thedearborntavern.com/,Chicago,IL
1,1,A&E Supply Co.,http://www.aesupplyco.com/,New York,NY
2,2,Cascabel,http://cascabelrestaurant.com/cms/,Los Angeles,CA
3,3,Sweetsalt,http://sweetsaltfood.com/,Los Angeles,CA
4,4,Ditmas,http://www.ditmasla.com/,Los Angeles,CA


In [20]:
    pd.read_sql_query('select * from chef_rest',con=engine).head()

,rest_id,chef_id
0,0,AaronCuschieri
1,1,AdamHarvey
2,2,AlexEusebio
3,3,AlexEusebio
4,4,AlexReznik


In [21]:
pd.read_sql_query('select * from season',con=engine).head()

,season_num,season_city
0,1,San Francisco
1,10,Seattle
2,11,New Orleans
3,12,Boston
4,13,California


In [22]:
#lets try running some more complicated queries.
session=Session(engine)
Base=automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['restaurants', 'chef_rest', 'season', 'contestants']

In [23]:
#connect to tables
c=Base.classes.contestants
cr=Base.classes.chef_rest
r=Base.classes.restaurants
s=Base.classes.season

In [24]:
#Print Top Chefs that have restaurants in NYC
NYC_rest=session.query(r.rest_name, r.rest_city, r.rest_state, c.chef_name, c.season_num).filter(r.rest_id==cr.rest_id).filter(cr.chef_id==c.chef_id).filter(r.rest_city=="New York").all()

for rest in NYC_rest:
     print (f"{rest.chef_name} from Season {rest.season_num} and owns {rest.rest_name} in {rest.rest_city}, {rest.rest_state}.")

Adam Harvey from Season 12 and owns A&E Supply Co. in New York, NY.
Alex Reznik from Season 7 and owns Bedford Kitchen in New York, NY.
Angelo Sosa from Season 7 and owns Hell’s Kitchen in New York, NY.
Angelo Sosa from Season 8 and owns Hell’s Kitchen in New York, NY.
Angelo Sosa from Season 7 and owns Añejo Tribeca in New York, NY.
Angelo Sosa from Season 8 and owns Añejo Tribeca in New York, NY.
Ash Fulk from Season 6 and owns Hill Country BBQ in New York, NY.
Ashley Merriman from Season 6 and owns The Waverly Inn in New York, NY.
Camille Becerra from Season 3 and owns De Maria in New York, NY.
Carla Hall from Season 5 and owns Carla Hall’s Southern Chicken in New York, NY.
Carla Hall from Season 8 and owns Carla Hall’s Southern Chicken in New York, NY.
Dale Talde from Season 4 and owns Massoni in New York, NY.
Dale Talde from Season 8 and owns Massoni in New York, NY.
Dale Talde from Season 4 and owns Talde in New York, NY.
Dale Talde from Season 8 and owns Talde in New York, NY.
F

In [25]:

winners=session.query(r.rest_name, r.rest_city, r.rest_state, c.chef_name, c.season_num).filter(r.rest_id==cr.rest_id).filter(cr.chef_id==c.chef_id).filter(c.ranked=="Winner").all()

for winner in winners:
     print (f"{winner.chef_name} won Top Chef Season {winner.season_num} and owns {winner.rest_name} in {winner.rest_city}, {winner.rest_state}.")

Hosea Rosenberg won Top Chef Season 5 and owns Blackbelly Market in Boulder, CO.
Jeremy Ford won Top Chef Season 13 and owns Matador Room in Miami, FL.
Kevin Sbraga won Top Chef Season 7 and owns The Fat Ham in King of Prussia, PA.
Michael Voltaggio won Top Chef Season 6 and owns Ink in Los Angeles, CA.
Michael Voltaggio won Top Chef Season 6 and owns Ink Sack in Los Angeles, CA.
Nicholas Elmi won Top Chef Season 11 and owns Laurel in Philadelphia, PA.
Paul Qui won Top Chef Season 9 and owns East Side King in Austin, TX.
Paul Qui won Top Chef Season 9 and owns Kuneho in Austin, TX.
Paul Qui won Top Chef Season 9 and owns Otoko in Austin, TX.
Paul Qui won Top Chef Season 9 and owns Thaikun in Austin, TX.
Paul Qui won Top Chef Season 9 and owns Pao by Paul Qui in Miami, FL.
Richard Blais won Top Chef Season 8 and owns Flip in Atlanta, GA.
Richard Blais won Top Chef Season 8 and owns Flip in Birmingham, AL.
Richard Blais won Top Chef Season 8 and owns Flip in Nashville, TN.
Richard Blais 

In [26]:
#Print Top Chefs that have restaurants in any city of your choosing
city=input("What city would you like to visit?")
any_rest=session.query(r.rest_name, r.rest_city, r.rest_state, c.chef_name, c.season_num).filter(r.rest_id==cr.rest_id).filter(cr.chef_id==c.chef_id).filter(r.rest_city==city).all()

for rest in any_rest:
     print (f"{rest.chef_name} from Season {rest.season_num} and owns {rest.rest_name} in {rest.rest_city}, {rest.rest_state}.")

What city would you like to visit?Atlanta
Eli Kirshtein from Season 6 and owns The Luminary in Atlanta, GA.
Hector Santiago from Season 6 and owns El Super Pan in Atlanta, GA.
Kevin Gillespie from Season 6 and owns Gunshow in Atlanta, GA.
Kevin Gillespie from Season 6 and owns Revival in Atlanta, GA.
Richard Blais from Season 4 and owns Flip in Atlanta, GA.
Richard Blais from Season 8 and owns Flip in Atlanta, GA.
Ron Eyester from Season 12 and owns Southern Bistro in Atlanta, GA.
